In [ ]:
# | default_exp spice.kernels

# SPICE Kernels
> Tools to manage SPICE kernels

## Intro

Feature list for this module:

* Receive the list of supported datasets for automatic retrieval of archived SPICE kernels
  * The supported datasets are tabled here at NAIF: https://naif.jpl.nasa.gov/naif/data_archived.html
* Receive the list of required SPICE kernels for a given mission and time range
* Automatic download of kernels for a given mission and time range either into a given location or the `planetarypy` local archive.

As always in `planetarypy` the general design philosophy is to first develop a management class to give the user full control over all the details, and then add easy-to-use function for the end-user that do the most frequently used things in one go. (See section "User Functions")

In [ ]:
# | export
import zipfile
from datetime import timedelta
from io import BytesIO
from itertools import repeat
from multiprocessing import cpu_count
from pathlib import Path

import hvplot.pandas  # noqa
import pandas as pd
import requests
import spiceypy as spice
from astropy.time import Time
from dask.distributed import Client
from fastcore.test import test_fail
from fastcore.utils import store_attr
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from yarl import URL

from planetarypy.config import config
from planetarypy.utils import nasa_time_to_iso, url_retrieve

In [ ]:
# | hide
from nbdev.showdoc import show_doc

In [ ]:
# | export
KERNEL_STORAGE = config.storage_root / "spice_kernels"
KERNEL_STORAGE.mkdir(exist_ok=True, parents=True)

In [ ]:
# | export
NAIF_URL = URL("https://naif.jpl.nasa.gov")
BASE_URL = NAIF_URL / "cgi-bin/subsetds.pl"

## Identifying and downloading kernel sets

One repeating task for SPICE calculations is the identification and retrieval of all SPICE kernels for a mission for a given time interval.

The folks at NAIF offer a "Subset" feature at their servers.
Here we set up a table of the currently supported datasets.

In [ ]:
# | export
datasets_url = "https://raw.githubusercontent.com/planetarypy/planetarypy_configs/main/archived_spice_kernel_sets.csv"

In [ ]:
# | export
datasets = pd.read_csv(datasets_url).set_index("shorthand")

In [ ]:
datasets

,path,Mission Name,PDS3 or PDS4,Data Size (GB),Start Time,Stop Time
shorthand,,,,,,
bc,bc/bc_spice,BepiColombo,4,2.6,2018-10-20,2024-01-01
cassini,co-s_j_e_v-spice-6-v1.0/cosp_1000,Cassini Orbiter,3,73.5,1997-10-15,2017-09-15
clementine,clem1-l-spice-6-v1.0/clsp_1000,Clementine,3,0.8,1994-01-26,1994-05-07
dart,dart/dart_spice,DART,4,10.1,2021-11-09,2050-01-01
dawn,dawn-m_a-spice-6-v1.0/dawnsp_1000,DAWN,3,86.4,2007-09-27,2018-10-31
di,di-c-spice-6-v1.0/disp_1000,Deep Impact,3,0.7,2005-01-12,2005-08-09
ds1,ds1-a_c-spice-6-v1.0/ds1sp_1000,Deep Space 1,3,0.9,1998-10-24,2001-12-18
epoxi,dif-c_e_x-spice-6-v1.0/epxsp_1000,EPOXI,3,1.0,2005-08-23,2011-03-01
em16,em16/em16_spice,ExoMars TGO 2016,4,11.0,2016-03-14,2024-01-01


To receive this dataframe:

```python
from planetarypy.spice.kernels import datasets
```
Some validation helpers:

In [ ]:
# | export
def is_start_valid(
    mission: str,  # mission shorthand label of datasets dataframe
    start: Time,  # start time in astropy.Time format
):
    return Time(datasets.at[mission, "Start Time"]) <= start


def is_stop_valid(
    mission: str,  # mission shorthand label of datasets dataframe
    stop: Time,  # stop time in astropy.Time format
):
    return Time(datasets.at[mission, "Stop Time"]) >= stop

In [ ]:
assert is_start_valid("cassini", Time("1998-01-01")) is True
assert is_start_valid("cassini", Time("1997-01-01")) is False
assert is_stop_valid("cassini", "2017-01-01") is True
assert is_stop_valid("cassini", "2018-01-01") is False

Now we build a management class for wrapping the Perl script available at below's URL for accessing subsets of these datasets.

First, the basic URLs we will use:

The Perl script `subsetds.pl` (the name at the end of the `BASE_URL`) requires as input:

* the dataset name
* start and stop of the time interval
* a constant named "Subset" to identify the action for this Perl script

We can assemble these parameters into a payload dictionary for the `requests.get` call and we manage different potential actions on the zipfile with a `Subsetter` class, that only requires the mission identifier, start and stop as parameters.

In [ ]:
# | export


def download_one_url(url, local_path, overwrite: bool = False):
    if local_path.exists() and not overwrite:
        return
    local_path.parent.mkdir(exist_ok=True, parents=True)
    url_retrieve(url, local_path)


class Subsetter:
    """Class to manage retrieving subset SPICE kernel lists

    Attributes
    ----------
    kernel_names: List of
    """

    def __init__(
        self,
        mission: str,  # mission shorthand in datasets dataframe
        start: str,  # start time in either ISO or yyyy-jjj format
        stop=None,  # stop time in either ISO or yyyy-jjj format
        save_location=None,  # overwrite default storing in planetarpy archive
    ):
        store_attr()
        self.initialize()

    def initialize(self):
        r = self.r
        if r.ok:
            z = zipfile.ZipFile(BytesIO(r.content))
        else:
            raise IOError("SPICE Server request returned status code: {r.status_code}")
        self.z = z
        # these files only exist "virtually" in the zip object, but are needed to
        # extract them:
        self.urls_file = [n for n in z.namelist() if n.startswith("urls_")][0]
        self.metakernel_file = [n for n in z.namelist() if n.lower().endswith(".tm")][0]
        with self.z.open(self.urls_file) as f:
            self.kernel_urls = f.read().decode().split()

    @property
    def r(self):
        return requests.get(BASE_URL, params=self.payload, stream=True)

    @property
    def start(self):
        return self._start

    @start.setter
    def start(self, value):
        try:
            self._start = Time(value)
        except ValueError:
            self._start = Time(nasa_time_to_iso(value))

    @property
    def stop(self):
        return self._stop

    @stop.setter
    def stop(self, value):
        if not value:
            self._stop = self.start + timedelta(days=1)
        else:
            try:
                self._stop = Time(value)
            except ValueError:
                self._stop = Time(nasa_time_to_iso(value))

    @property
    def payload(self):
        """Put payload together while checking for working time format.

        If Time(self.start) doesn't work, then we assume that the date must be in the
        Time-unsupported yyyy-jjj format, which can be converted by `nasa_time_to_iso`
        from `planetarypy.utils`.
        """
        if not (
            is_start_valid(self.mission, self.start)
            and is_stop_valid(self.mission, self.stop)
        ):
            raise ValueError(
                "One of start/stop is outside the supported date-range. See `datasets`."
            )
        p = {
            "dataset": datasets.loc[self.mission, "path"],
            "start": self.start.iso,
            "stop": self.stop.iso,
            "action": "Subset",
        }
        return p

    @property
    def kernel_names(self):
        "Return list of names of kernels for the given time range."
        return [
            str(Path(URL(url).parent.name) / URL(url).name) for url in self.kernel_urls
        ]

    def get_local_path(
        self,
        url,  # kernel url to determine local storage path
    ) -> Path:  # full local path where kernel in URL will be stored
        """Calculate local storage path from Kernel URL, using `save_location` if given.

        If self.save_location is None, the `planetarypy` archive is being used.
        """
        u = URL(url)
        basepath = (
            KERNEL_STORAGE / self.mission
            if not self.save_location
            else self.save_location
        )
        return basepath / u.parent.name / u.name

    def _non_blocking_download(self, overwrite: bool = False):
        with Client() as client:
            futures = []
            for url in tqdm(self.kernel_urls, desc="Kernels downloaded"):
                local_path = self.get_local_path(url)
                if local_path.exists() and not overwrite:
                    print(local_path.parent.name, local_path.name, "locally available.")
                    continue
                local_path.parent.mkdir(exist_ok=True, parents=True)
                futures.append(client.submit(url_retrieve, url, local_path))
            return [f.result() for f in futures]

    def _concurrent_download(self, overwrite: bool = False):
        paths = [self.get_local_path(url) for url in self.kernel_urls]
        args = zip(self.kernel_urls, paths, repeat(overwrite))
        results = process_map(download_one_url, args, max_workers=cpu_count() - 2)

    def download_kernels(
        self,
        overwrite: bool = False,  # switch to control if kernels should be downloaded over existing ones
        non_blocking: bool = False,
        quiet: bool = False,
    ):
        if non_blocking:
            return self._non_blocking_download(overwrite)
        # sequential download
        for url in tqdm(self.kernel_urls, desc="Kernels downloaded"):
            local_path = self.get_local_path(url)
            if local_path.exists() and not overwrite:
                if not quiet:
                    print(local_path.parent.name, local_path.name, "locally available.")
                continue
            local_path.parent.mkdir(exist_ok=True, parents=True)
            url_retrieve(url, local_path)

    def get_metakernel(self) -> Path:  # return path to metakernel file
        """Get metakernel file from NAIF and adapt path to match local storage.

        Use `save_location` if given, otherwise `planetarypy` archive.
        """
        basepath = (
            KERNEL_STORAGE / self.mission
            if not self.save_location
            else self.save_location
        )
        savepath = basepath / self.metakernel_file
        with open(savepath, "w") as outfile, self.z.open(
            self.metakernel_file
        ) as infile:
            for line in infile:
                linestr = line.decode()
                if "'./data'" in linestr:
                    linestr = linestr.replace("'./data'", f"'{savepath.parent}'")
                outfile.write(linestr)
        return savepath

In [ ]:
subset = Subsetter("cassini", "2014-270")

In [ ]:
subset.kernel_names

['ck/14001_15001pa_gapfill_v14.bc',
 'ck/14212_14279py_as_flown.bc',
 'ck/14268_14273ra.bc',
 'ck/cas_cda_20150318.bc',
 'ck/cas_lemms_05109_20001_v2.bc',
 'dsk/cas_enceladus_ssd_spc_0128icq_v1.bds',
 'fk/cas_dyn_v03.tf',
 'fk/cas_mimi_v202.tf',
 'fk/cas_rocks_v18.tf',
 'fk/cas_v43.tf',
 'ik/cas_caps_v03.ti',
 'ik/cas_cda_v01.ti',
 'ik/cas_cirs_v10.ti',
 'ik/cas_inms_v02.ti',
 'ik/cas_iss_v10.ti',
 'ik/cas_mag_v01.ti',
 'ik/cas_mimi_v11.ti',
 'ik/cas_radar_v11.ti',
 'ik/cas_rpws_v01.ti',
 'ik/cas_rss_v03.ti',
 'ik/cas_sru_v02.ti',
 'ik/cas_uvis_v07.ti',
 'ik/cas_vims_v06.ti',
 'lsk/naif0012.tls',
 'pck/enceladus_ssd_230702_v1.tpc',
 'pck/pck00010.tpc',
 'sclk/cas00172.tsc',
 'spk/140809BP_IRRE_00256_25017.bsp',
 'spk/180927AP_RE_90165_18018.bsp',
 'spk/200128RU_SCPSE_14251_14283.bsp',
 'spk/sat427.bsp']

In [ ]:
len(subset.kernel_names)

31

In [ ]:
# this should fail:
def _failing():
    Subsetter("cassini", "2019-01-01")


test_fail(_failing, contains="start/stop")

In [ ]:
subset = Subsetter("cassini", "2011-02-13", "2011-02-14")

In [ ]:
subset.urls_file

'urls_cosp_1000_110213_110214.txt'

In [ ]:
subset.metakernel_file

'cas_2011_v18_110213_110214.tm'

In [ ]:
show_doc(Subsetter.kernel_names)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L154){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.kernel_names

>      Subsetter.kernel_names ()

*Return list of names of kernels for the given time range.*

In [ ]:
subset.kernel_names

['ck/11001_12001pa_gapfill_v14.bc',
 'ck/11017_11066py_as_flown.bc',
 'ck/11044_11049ra.bc',
 'ck/cas_caps_11032_11060_v2.bc',
 'ck/cas_cda_20120517.bc',
 'ck/cas_lemms_05109_20001_v2.bc',
 'dsk/cas_enceladus_ssd_spc_0128icq_v1.bds',
 'fk/cas_dyn_v03.tf',
 'fk/cas_mimi_v202.tf',
 'fk/cas_rocks_v18.tf',
 'fk/cas_v43.tf',
 'ik/cas_caps_v03.ti',
 'ik/cas_cda_v01.ti',
 'ik/cas_cirs_v10.ti',
 'ik/cas_inms_v02.ti',
 'ik/cas_iss_v10.ti',
 'ik/cas_mag_v01.ti',
 'ik/cas_mimi_v11.ti',
 'ik/cas_radar_v11.ti',
 'ik/cas_rpws_v01.ti',
 'ik/cas_rss_v03.ti',
 'ik/cas_sru_v02.ti',
 'ik/cas_uvis_v07.ti',
 'ik/cas_vims_v06.ti',
 'lsk/naif0012.tls',
 'pck/enceladus_ssd_230702_v1.tpc',
 'pck/pck00010.tpc',
 'sclk/cas00172.tsc',
 'spk/140809BP_IRRE_00256_25017.bsp',
 'spk/180927AP_RE_90165_18018.bsp',
 'spk/200128RU_SCPSE_11041_11093.bsp',
 'spk/sat427.bsp']

In [ ]:
subset.kernel_urls

['https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/11001_12001pa_gapfill_v14.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/11017_11066py_as_flown.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/11044_11049ra.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/cas_caps_11032_11060_v2.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/cas_cda_20120517.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/cas_lemms_05109_20001_v2.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/dsk/cas_enceladus_ssd_spc_0128icq_v1.bds',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/fk/cas_dyn_v03.tf',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/

In [ ]:
show_doc(Subsetter.get_local_path)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L160){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.get_local_path

>      Subsetter.get_local_path (url)

*Calculate local storage path from Kernel URL, using `save_location` if given.

If self.save_location is None, the `planetarypy` archive is being used.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| url |  | kernel url to determine local storage path |
| **Returns** | **Path** | **full local path where kernel in URL will be stored** |

In [ ]:
subset.get_local_path(subset.kernel_urls[0])

Path('/Users/maye/planetarypy_data/spice_kernels/cassini/ck/11001_12001pa_gapfill_v14.bc')

In [ ]:
subset.save_location = Path(".")

In [ ]:
subset.get_local_path(subset.kernel_urls[0])

Path('ck/11001_12001pa_gapfill_v14.bc')

In [ ]:
show_doc(Subsetter.download_kernels)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L193){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.download_kernels

>      Subsetter.download_kernels (overwrite:bool=False,
>                                  non_blocking:bool=False, quiet:bool=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| overwrite | bool | False | switch to control if kernels should be downloaded over existing ones |
| non_blocking | bool | False |  |
| quiet | bool | False |  |

In [ ]:
# reset save_location to prevent additional download
subset.save_location = None

In [ ]:
subset.download_kernels()

Kernels downloaded:   0%|          | 0/32 [00:00<?, ?it/s]

ck 11001_12001pa_gapfill_v14.bc locally available.
ck 11017_11066py_as_flown.bc locally available.
ck 11044_11049ra.bc locally available.
ck cas_caps_11032_11060_v2.bc locally available.
ck cas_cda_20120517.bc locally available.
ck cas_lemms_05109_20001_v2.bc locally available.
dsk cas_enceladus_ssd_spc_0128icq_v1.bds locally available.
fk cas_dyn_v03.tf locally available.
fk cas_mimi_v202.tf locally available.
fk cas_rocks_v18.tf locally available.
fk cas_v43.tf locally available.
ik cas_caps_v03.ti locally available.
ik cas_cda_v01.ti locally available.
ik cas_cirs_v10.ti locally available.
ik cas_inms_v02.ti locally available.
ik cas_iss_v10.ti locally available.
ik cas_mag_v01.ti locally available.
ik cas_mimi_v11.ti locally available.
ik cas_radar_v11.ti locally available.
ik cas_rpws_v01.ti locally available.
ik cas_rss_v03.ti locally available.
ik cas_sru_v02.ti locally available.
ik cas_uvis_v07.ti locally available.
ik cas_vims_v06.ti locally available.
lsk naif0012.tls locall

In [ ]:
subset.download_kernels(non_blocking=True)

Kernels downloaded:   0%|          | 0/32 [00:00<?, ?it/s]

ck 11001_12001pa_gapfill_v14.bc locally available.
ck 11017_11066py_as_flown.bc locally available.
ck 11044_11049ra.bc locally available.
ck cas_caps_11032_11060_v2.bc locally available.
ck cas_cda_20120517.bc locally available.
ck cas_lemms_05109_20001_v2.bc locally available.
dsk cas_enceladus_ssd_spc_0128icq_v1.bds locally available.
fk cas_dyn_v03.tf locally available.
fk cas_mimi_v202.tf locally available.
fk cas_rocks_v18.tf locally available.
fk cas_v43.tf locally available.
ik cas_caps_v03.ti locally available.
ik cas_cda_v01.ti locally available.
ik cas_cirs_v10.ti locally available.
ik cas_inms_v02.ti locally available.
ik cas_iss_v10.ti locally available.
ik cas_mag_v01.ti locally available.
ik cas_mimi_v11.ti locally available.
ik cas_radar_v11.ti locally available.
ik cas_rpws_v01.ti locally available.
ik cas_rss_v03.ti locally available.
ik cas_sru_v02.ti locally available.
ik cas_uvis_v07.ti locally available.
ik cas_vims_v06.ti locally available.
lsk naif0012.tls locall

[]

In [ ]:
show_doc(Subsetter.get_metakernel)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L211){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.get_metakernel

>      Subsetter.get_metakernel ()

*Get metakernel file from NAIF and adapt path to match local storage.

Use `save_location` if given, otherwise `planetarypy` archive.*

In [ ]:
mkpath = subset.get_metakernel()
mkpath

Path('/Users/maye/planetarypy_data/spice_kernels/cassini/cas_2011_v18_110213_110214.tm')

In [ ]:
!cat {mkpath}

KPL/MK

   This meta-kernel lists a subset of kernels from the meta-kernel
   cas_2011_v18.tm provided in the CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive,
   covering the whole or a part of the customer requested time period
   from 2011-02-13T00:00:00.000 to 2011-02-14T00:00:00.000.

   The documentation describing these kernels can be found in the
   complete CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive available at this URL

   https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000

   To use this meta-kernel users may need to modify the value of the
   PATH_VALUES keyword to point to the actual location of the archive's
   ``data'' directory on their system. Replacing ``/'' with ``\''
   and converting line terminators to the format native to the user's
   system may also be required if this meta-kernel is to be used on a
   non-UNIX workstation.

   This meta-kernel was created by the NAIF node's SPICE PDS archive
   subsetting service version 2.1 on Wed Sep 25

Loading the metakernel works! :

In [ ]:
spice.furnsh(str(mkpath))

Or, with given `save_location`:

In [ ]:
subset.save_location = Path(".")
mkpath = subset.get_metakernel()
mkpath

Path('cas_2011_v18_110213_110214.tm')

The metakernel is correctly adapted, however for these tests, I didn't download the kernels again

In [ ]:
!cat {mkpath}

KPL/MK

   This meta-kernel lists a subset of kernels from the meta-kernel
   cas_2011_v18.tm provided in the CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive,
   covering the whole or a part of the customer requested time period
   from 2011-02-13T00:00:00.000 to 2011-02-14T00:00:00.000.

   The documentation describing these kernels can be found in the
   complete CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive available at this URL

   https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000

   To use this meta-kernel users may need to modify the value of the
   PATH_VALUES keyword to point to the actual location of the archive's
   ``data'' directory on their system. Replacing ``/'' with ``\''
   and converting line terminators to the format native to the user's
   system may also be required if this meta-kernel is to be used on a
   non-UNIX workstation.

   This meta-kernel was created by the NAIF node's SPICE PDS archive
   subsetting service version 2.1 on Wed Sep 25

## User functions

In [ ]:
# | export
def get_metakernel_and_files(
    mission: str,  # mission shorthand from datasets dataframe
    start: str,  # start time as iso-string, or yyyy-jjj
    stop: str,  # stop time as iso-string or yyyy-jjj
    save_location: str = None,  # override storage into planetarypy archive
    quiet: bool = False,  # suppress kernel-wise feedback
) -> Path:  # pathlib.Path to metakernel file with corrected data path.
    "For a given mission and start/stop times, download the kernels and get metakernel path"
    subset = Subsetter(mission, start, stop, save_location)
    subset.download_kernels(non_blocking=True, quiet=quiet)
    return subset.get_metakernel()

In [ ]:
mkpath = get_metakernel_and_files("cassini", "2011-02-13", "2011-02-14")
mkpath

Kernels downloaded:   0%|          | 0/32 [00:00<?, ?it/s]

ck 11001_12001pa_gapfill_v14.bc locally available.
ck 11017_11066py_as_flown.bc locally available.
ck 11044_11049ra.bc locally available.
ck cas_caps_11032_11060_v2.bc locally available.
ck cas_cda_20120517.bc locally available.
ck cas_lemms_05109_20001_v2.bc locally available.
dsk cas_enceladus_ssd_spc_0128icq_v1.bds locally available.
fk cas_dyn_v03.tf locally available.
fk cas_mimi_v202.tf locally available.
fk cas_rocks_v18.tf locally available.
fk cas_v43.tf locally available.
ik cas_caps_v03.ti locally available.
ik cas_cda_v01.ti locally available.
ik cas_cirs_v10.ti locally available.
ik cas_inms_v02.ti locally available.
ik cas_iss_v10.ti locally available.
ik cas_mag_v01.ti locally available.
ik cas_mimi_v11.ti locally available.
ik cas_radar_v11.ti locally available.
ik cas_rpws_v01.ti locally available.
ik cas_rss_v03.ti locally available.
ik cas_sru_v02.ti locally available.
ik cas_uvis_v07.ti locally available.
ik cas_vims_v06.ti locally available.
lsk naif0012.tls locall

Path('/Users/maye/planetarypy_data/spice_kernels/cassini/cas_2011_v18_110213_110214.tm')

In [ ]:
# | export
def list_kernels_for_day(
    mission: str,  # mission shorthand from datasets dataframe
    start: str,  # start time as iso-string, or yyyy-jjj
    stop: str = "",  # stop time as iso-string or yyyy-jjj
) -> list:  # list of kernel names
    subset = Subsetter(mission, start, stop)
    return subset.kernel_names

In [ ]:
list_kernels_for_day("mro", "2015-02-13")

['ck/mro_crm_psp_150201_150228.bc',
 'ck/mro_hga_psp_150210_150216_v2.bc',
 'ck/mro_mcs_psp_150201_150228.bc',
 'ck/mro_sa_psp_150210_150216_v2.bc',
 'ck/mro_sc_psp_150210_150216_v2.bc',
 'fk/mro_v16.tf',
 'ik/mro_crism_v10.ti',
 'ik/mro_ctx_v11.ti',
 'ik/mro_hirise_v12.ti',
 'ik/mro_marci_v10.ti',
 'ik/mro_mcs_v10.ti',
 'ik/mro_onc_v10.ti',
 'lsk/naif0012.tls',
 'pck/pck00008.tpc',
 'sclk/mro_sclkscet_00095_65536.tsc',
 'spk/de421.bsp',
 'spk/mar097.bsp',
 'spk/mro_psp34.bsp',
 'spk/mro_psp34_ssd_mro95a.bsp',
 'spk/mro_struct_v10.bsp']

In [ ]:
# this always takes much longer than other requests!
list_kernels_for_day("maven", "2017-01-01")

['ck/mvn_app_rel_161226_170101_v01.bc',
 'ck/mvn_app_rel_170102_170108_v01.bc',
 'ck/mvn_iuvs_rem_170101_170331_v03.bc',
 'ck/mvn_sc_rel_161226_170101_v01.bc',
 'ck/mvn_sc_rel_170102_170108_v02.bc',
 'ck/mvn_swea_nom_131118_300101_v02.bc',
 'fk/maven_v11.tf',
 'ik/maven_ant_v10.ti',
 'ik/maven_euv_v10.ti',
 'ik/maven_iuvs_v11.ti',
 'ik/maven_ngims_v10.ti',
 'ik/maven_sep_v12.ti',
 'ik/maven_static_v11.ti',
 'ik/maven_swea_v11.ti',
 'ik/maven_swia_v10.ti',
 'lsk/naif0012.tls',
 'pck/pck00010.tpc',
 'sclk/mvn_sclkscet_00112.tsc',
 'spk/de430s.bsp',
 'spk/mar097s.bsp',
 'spk/maven_orb_rec_170101_170401_v1.bsp',
 'spk/maven_struct_v01.bsp']

In [ ]:
# this always takes much longer than other requests!
list_kernels_for_day("orex", "2018-11-30")

['ck/orx_sa_rel_181126_181202_v01.bc',
 'ck/orx_sc_rel_181126_181202_v01.bc',
 'ck/orx_struct_mapcam_v01.bc',
 'ck/orx_struct_polycam_v01.bc',
 'dsk/bennu_g_00880mm_alt_obj_0000n00000_v021a.bds',
 'fk/orx_shape_v03.tf',
 'fk/orx_v14.tf',
 'ik/orx_lidar_v00.ti',
 'ik/orx_navcam_v02.ti',
 'ik/orx_ocams_v07.ti',
 'ik/orx_ola_v01.ti',
 'ik/orx_otes_v00.ti',
 'ik/orx_ovirs_v00.ti',
 'ik/orx_rexis_v01.ti',
 'ik/orx_stowcam_v00.ti',
 'ik/orx_struct_v00.ti',
 'lsk/naif0012.tls',
 'pck/bennu_v17.tpc',
 'pck/pck00010.tpc',
 'sclk/orx_sclkscet_00093.tsc',
 'spk/bennu_refdrmc_v1.bsp',
 'spk/de424.bsp',
 'spk/orx_180301_181201_180921_od044_v1.bsp',
 'spk/orx_180801_190302_181218_od077_v1.bsp',
 'spk/orx_181203_190302_190104_od085_v1.bsp',
 'spk/orx_struct_v04.bsp']

In [ ]:
# |filter_stream ErfaWarning
# |eval: false

import time


def _test_mission_kernels_available(mission):
    print("Doing", mission)
    start = datasets.at[mission, "Start Time"]
    end = datasets.at[mission, "Stop Time"]
    half = Time(start) + (Time(end) - Time(start)) / 2
    print("Half time:", half)
    t0 = time.time()
    try:
        found = list_kernels_for_day(mission, half)
    except IndexError:
        print("Problem with", mission)
    else:
        print(f"Found {len(found)} kernels for {mission}")
        print("Time taken:", t := time.time() - t0)
    return dict(mission=mission, processing_time=t, n_kernels=len(found))

In [ ]:
timings = []
for mission in datasets.index:
    timings.append(_test_mission_kernels_available(mission))

Doing bc
Half time: 2021-05-26 12:00:00.000
Found 53 kernels for bc
Time taken: 2.1192197799682617
Doing cassini
Half time: 2007-09-30 12:00:01.000
Found 36 kernels for cassini
Time taken: 1.2332758903503418
Doing clementine
Half time: 1994-03-17 12:00:00.000
Found 19 kernels for clementine
Time taken: 0.8816888332366943
Doing dart
Half time: 2035-12-06 00:00:00.000


/Users/maye/miniforge3/envs/py312/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/maye/miniforge3/envs/py312/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/maye/miniforge3/envs/py312/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/maye/miniforge3/envs/py312/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


Found 16 kernels for dart
Time taken: 1.444464921951294
Doing dawn
Half time: 2013-04-14 00:00:00.000
Found 21 kernels for dawn
Time taken: 1.12506103515625
Doing di
Half time: 2005-04-26 12:00:00.000
Found 16 kernels for di
Time taken: 0.8815557956695557
Doing ds1
Half time: 2000-05-21 11:59:59.500
Found 11 kernels for ds1
Time taken: 0.8857688903808594
Doing epoxi
Half time: 2008-05-27 00:00:00.000
Found 12 kernels for epoxi
Time taken: 0.9109499454498291
Doing em16
Half time: 2020-02-06 11:59:59.500
Found 39 kernels for em16
Time taken: 2.2251791954040527
Doing grail
Half time: 2012-04-29 00:00:00.500
Found 20 kernels for grail
Time taken: 0.9258971214294434
Doing hayabusa
Half time: 2005-10-15 12:00:00.000
Found 15 kernels for hayabusa
Time taken: 1.0151360034942627
Doing insight
Half time: 2020-08-24 12:00:00.000
Found 21 kernels for insight
Time taken: 1.842237949371338
Doing juno
Half time: 2017-08-01 23:59:58.500
Found 23 kernels for juno
Time taken: 1.2264678478240967
Doing la

In [ ]:
df = pd.DataFrame(timings).sort_values("processing_time", ascending=False)

In [ ]:
plot = df.hvplot.scatter(
    x="n_kernels", y="processing_time", by="mission", width=600, height=400
)
plot = plot.opts(legend_position="bottom", legend_cols=6)
plot

:NdOverlay   [mission]
   :Scatter   [n_kernels]   (processing_time)

> NOTE: Any ErfaWarnings above are caused by the LADEE mission using a kernel up to 2050, and the astropy.Time module warns about potential precicision issues regarding unknown leapseconds that will be put in in the future.

### Generic kernel management

There are a few generic kernels that are required for basic illumination calculations as supported by this package.

In [ ]:
# | export
GENERIC_STORAGE = KERNEL_STORAGE / "generic"
GENERIC_STORAGE.mkdir(exist_ok=True, parents=True)
GENERIC_URL = NAIF_URL / "pub/naif/generic_kernels/"

generic_kernel_names = [
    "lsk/naif0012.tls",
    "pck/pck00010.tpc",
    "pck/de-403-masses.tpc",
    "spk/planets/de430.bsp",
    # "spk/satellites/mar097.bsp",
]
generic_kernel_paths = [GENERIC_STORAGE.joinpath(i) for i in generic_kernel_names]

In [ ]:
# | export


def download_generic_kernels(overwrite=False):
    "Download all kernels as required by generic_kernel_list."
    dl_urls = [GENERIC_URL / i for i in generic_kernel_names]
    for dl_url, savepath in zip(dl_urls, generic_kernel_paths):
        if savepath.exists() and not overwrite:
            print(
                savepath.name,
                "already downloaded. Use `overwrite=True` to download again.",
            )
            continue
        savepath.parent.mkdir(exist_ok=True, parents=True)
        url_retrieve(dl_url, savepath)

In [ ]:
download_generic_kernels()

naif0012.tls already downloaded. Use `overwrite=True` to download again.
pck00010.tpc already downloaded. Use `overwrite=True` to download again.
de-403-masses.tpc already downloaded. Use `overwrite=True` to download again.
de430.bsp already downloaded. Use `overwrite=True` to download again.


In [ ]:
# | export


def load_generic_kernels():
    """Load all kernels in generic_kernels list.

    Loads pure planetary bodies meta-kernel without spacecraft data.

    Downloads any missing generic kernels.
    """
    if any([not p.exists() for p in generic_kernel_paths]):
        download_generic_kernels()
    for kernel in generic_kernel_paths:
        spice.furnsh(str(kernel))

In [ ]:
spice.kclear()

In [ ]:
load_generic_kernels()

In [ ]:
# | export


def show_loaded_kernels():
    "Print overview of loaded kernels."
    count = spice.ktotal("all")
    if count == 0:
        print("No kernels loaded at this time.")
    else:
        print("The loaded files are:\n(paths relative to kernels.KERNEL_STORAGE)\n")
    for which in range(count):
        out = spice.kdata(which, "all", 100, 100, 100)
        print("Position:", which)
        p = Path(out[0])
        print("Path", p.relative_to(KERNEL_STORAGE))
        print("Type:", out[1])
        print("Source:", out[2])
        print("Handle:", out[3])
        # print("Found:", out[4])

In [ ]:
show_loaded_kernels()

The loaded files are:
(paths relative to kernels.KERNEL_STORAGE)

Position: 0
Path generic/lsk/naif0012.tls
Type: TEXT
Source: 
Handle: 0
Position: 1
Path generic/pck/pck00010.tpc
Type: TEXT
Source: 
Handle: 0
Position: 2
Path generic/pck/de-403-masses.tpc
Type: TEXT
Source: 
Handle: 0
Position: 3
Path generic/spk/planets/de430.bsp
Type: SPK
Source: 
Handle: 12


In [ ]:
from nbdev import nbdev_export

nbdev_export()